In [1]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import dill
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder
import category_encoders as ce
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss, precision_score, recall_score
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from helper_functions import *

In [2]:
df_test = load_data('application_test.csv')

In [3]:
df_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [4]:
df_test = drop_features(df_test, features_to_drop=['SK_ID_CURR'])

In [5]:
df_test

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [6]:
df_test = clean_data(df_test)

In [7]:
df_test.isna().sum().sum()

np.int64(0)

In [8]:
df_test = df_test.reset_index(drop=True)

In [9]:
# Get only categorical features:
categorical_cols = df_test.select_dtypes(include=['object']).columns.tolist()

df_test = encode_data(df_test, 'TARGET', categorical_cols, train=False,  model=TargetEncoder)


In [10]:
df_test

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0.083459,0.069993,0.085002,0.079616,0,135000.0,568800.0,20560.5,450000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.083459,0.101419,0.085002,0.079616,0,99000.0,222768.0,17370.0,180000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,0.083459,0.101419,0.072437,0.079616,0,202500.0,663264.0,69777.0,630000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,0.083459,0.069993,0.085002,0.079616,2,315000.0,1575000.0,49018.5,1575000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,0.083459,0.101419,0.072437,0.083249,1,180000.0,625500.0,32067.0,625500.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0.083459,0.069993,0.085002,0.079616,0,121500.0,412560.0,17473.5,270000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,0.083459,0.069993,0.085002,0.083249,2,157500.0,622413.0,31909.5,495000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
48741,0.083459,0.069993,0.072437,0.079616,1,202500.0,315000.0,33205.5,315000.0,0.081687,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,0.083459,0.101419,0.085002,0.083249,0,225000.0,450000.0,25128.0,450000.0,0.074946,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [11]:
df_test = scale_data(df_test, scaler=StandardScaler, target=None, features_to_scale=None)

In [12]:
df_test

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0.095331,-0.701175,0.713154,-0.669488,-0.559988,-0.427809,0.142475,-0.553580,-0.037477,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,-1.153865
1,0.095331,1.426178,0.713154,-0.669488,-0.559988,-0.782413,-0.804537,-0.752831,-0.839362,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.589324
2,0.095331,1.426178,-1.402222,-0.669488,-0.559988,0.237075,0.401002,2.520066,0.497113,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,0.773588,1.170387
3,0.095331,-0.701175,0.713154,-0.669488,2.260729,1.345214,2.896221,1.223666,3.303709,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.589324
4,0.095331,1.426178,-1.402222,1.493678,0.850370,0.015447,0.297651,0.165019,0.483748,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0.095331,-0.701175,0.713154,-0.669488,-0.559988,-0.560785,-0.285119,-0.746368,-0.572067,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,-0.572802
48740,0.095331,-0.701175,0.713154,1.493678,2.260729,-0.206181,0.289202,0.155183,0.096170,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261
48741,0.095331,-0.701175,-1.402222,-0.669488,0.850370,0.237075,-0.552119,0.236120,-0.438420,0.337608,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,3.743406,-0.572802
48742,0.095331,1.426178,0.713154,1.493678,-0.559988,0.458703,-0.182654,-0.268332,-0.037477,-2.250853,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261


In [13]:
with open('trained_logistic_model.pickle', 'rb') as f: 
    lr_model = dill.load(f)

y_new_pred = predict_model(df_test, lr_model)

In [14]:
y_new_pred

array([0, 1, 0, ..., 0, 0, 1])

In [15]:
with open('trained_random_forest_model.pickle', 'rb') as f: 
    rf_model = dill.load(f)

with open('splitted_train_test.pickle', 'rb') as f: 
    _, _, _, y_test = dill.load(f)

y_test = y_test.reset_index(drop=True)

y_new_pred1 = predict_model(df_test, rf_model)

In [16]:
y_new_pred1

array([0, 1, 0, ..., 0, 0, 1])

In [17]:
with open('trained_xgb_model.pickle', 'rb') as f: 
    xgb_model = dill.load(f)

y_new_pred2 = predict_model(df_test, xgb_model)

In [18]:
y_new_pred2

array([1, 1, 0, ..., 0, 0, 0])

In [21]:
# Ensure y_test and y_new_pred are of the same length 
common_indices = y_test.index.intersection(df_test.index) 
y_test_aligned = y_test.loc[common_indices] 
y_new_pred_aligned = pd.Series(y_new_pred, index=df_test.index).loc[common_indices]
y_new_pred_aligned1 = pd.Series(y_new_pred1, index=df_test.index).loc[common_indices]
y_new_pred_aligned2 = pd.Series(y_new_pred2, index=df_test.index).loc[common_indices]


In [22]:
if len(y_test_aligned) == len(y_new_pred_aligned): 
    lr_accuracy = accuracy_score(y_test_aligned.values.flatten(), y_new_pred_aligned) 
    lr_f1 = f1_score(y_test_aligned.values.flatten(), y_new_pred_aligned)
    lr_auc = roc_auc_score(y_test_aligned.values.flatten(), y_new_pred_aligned)

    rf_accuracy = accuracy_score(y_test_aligned.values.flatten(), y_new_pred_aligned1) 
    rf_f1 = f1_score(y_test_aligned.values.flatten(), y_new_pred_aligned1)
    rf_auc = roc_auc_score(y_test_aligned.values.flatten(), y_new_pred_aligned1)


    xgb_accuracy = accuracy_score(y_test_aligned.values.flatten(), y_new_pred_aligned2) 
    xgb_f1 = f1_score(y_test_aligned.values.flatten(), y_new_pred_aligned2)
    xgb_auc = roc_auc_score(y_test_aligned.values.flatten(), y_new_pred_aligned2)

    print(f"LR: Accuracy Score: {lr_accuracy}, F1_score: {lr_f1}, AUC: {lr_auc}") 
    print(f"RF: Accuracy Score: {rf_accuracy}, F1_score: {rf_f1}, AUC: {rf_auc}") 
    print(f"xgb: Accuracy Score: {xgb_accuracy}, F1_score: {xgb_f1}, AUC: {xgb_auc}") 

else: 
    print("Length mismatch: y_test_aligned has length", len(y_test_aligned), "while y_new_pred_aligned has length", len(y_new_pred_aligned))

LR: Accuracy Score: 0.6322214016084031, F1_score: 0.12819141175898457, AUC: 0.4974866812040203
RF: Accuracy Score: 0.6868537666174298, F1_score: 0.12174913693901036, AUC: 0.4968458559151958
xgb: Accuracy Score: 0.7996266207122928, F1_score: 0.10336913614247682, AUC: 0.5003885837576619
